### Import Library

In [1]:
# Mengaktifkan visualisasi inline matplotlib di notebook
get_ipython().run_line_magic('matplotlib', 'inline')

# Mengimpor library yang diperlukan
import pandas as pd              # Untuk manipulasi data berbentuk tabel
import numpy as np               # Untuk komputasi numerik
from textblob import TextBlob    # Untuk pemrosesan teks dan analisis sentimen
from sklearn.svm import SVC      # Support Vector Classification, algoritma untuk klasifikasi
import seaborn as sns            # Untuk visualisasi data yang lebih menarik
sns.set_context('notebook')
sns.set_style('white')
import nltk                      # Natural Language Toolkit untuk pemrosesan bahasa alami
from nltk.corpus import stopwords # Untuk menghilangkan kata-kata umum yang tidak bermakna
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer  # Untuk mengubah teks menjadi vektor fitur
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV  # Untuk pembagian data dan optimasi parameter
from sklearn.metrics import classification_report  # Untuk evaluasi model
from sklearn.pipeline import Pipeline  # Untuk menggabungkan beberapa langkah pemrosesan

### Load Data

In [2]:
# Membaca data dari file training.txt dengan pemisah tab
# Kolom diberi nama 'liked' dan 'text', dengan 'liked' sebagai label sentimen
df = pd.read_csv("training.txt", sep="\t", names=['liked', 'text'], encoding="utf-8")
df.head()  # Menampilkan 5 data pertama

,liked,text
0,1,India is developing countries
1,1,The Da Vinci Code book is just awesome.
2,1,this was the first clive cussler i've ever rea...
3,1,i liked the Da Vinci Code a lot.
4,1,i liked the Da Vinci Code a lot.


### Exploratory Data Analysis (EDA)

In [3]:
# Menampilkan jumlah total data dalam dataset
print(len(df))

6931


In [4]:
# Menampilkan statistik deskriptif berdasarkan kategori 'liked'
df.groupby('liked').describe()

text                                  
      count unique                   top freq
liked                                        
0      2975    559  I hate Harry Potter.   85
1      3956    744  I love Harry Potter.  167

### Data Preprocessing

In [5]:
# Fungsi untuk mengekstrak token (kata-kata) dari teks
def tokens(review):
    return TextBlob(review).words


In [6]:
# Menerapkan fungsi tokens pada 5 data pertama
df.head().text.apply(tokens)

0                   [India, is, developing, countries]
1      [The, Da, Vinci, Code, book, is, just, awesome]
2    [this, was, the, first, clive, cussler, i, 've...
3             [i, liked, the, Da, Vinci, Code, a, lot]
4             [i, liked, the, Da, Vinci, Code, a, lot]
Name: text, dtype: object

In [7]:
# Contoh penggunaan TextBlob untuk POS tagging (menandai jenis kata)
TextBlob("ready was not a good movie").tags
#nltk.help.upenn_tagset('JJ')  # Untuk melihat penjelasan tag JJ (adjective)

[('ready', 'NN'),
 ('was', 'VBD'),
 ('not', 'RB'),
 ('a', 'DT'),
 ('good', 'JJ'),
 ('movie', 'NN')]

In [8]:
# Fungsi untuk mengubah kata-kata ke bentuk dasar (lemmatization)
def to_lemmas(review):
    wordss = TextBlob(review.lower()).words
    # for each word, take its "base form" = lemma
    return [word.lemma for word in wordss]

# Menerapkan fungsi to_lemmas pada 5 data pertama
df.text.head().apply(to_lemmas)

0                     [india, is, developing, country]
1      [the, da, vinci, code, book, is, just, awesome]
2    [this, wa, the, first, clive, cussler, i, 've,...
3             [i, liked, the, da, vinci, code, a, lot]
4             [i, liked, the, da, vinci, code, a, lot]
Name: text, dtype: object

In [9]:
# Contoh lemmatization menggunakan WordNetLemmatizer
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
lmtzr.lemmatize('octopi')  # Mengubah bentuk jamak 'octopi' menjadi bentuk tunggal 'octopus'
#nltk

'octopus'

Convert data ke vectors

In [10]:
# Membuat transformer Bag of Words dan melatihnya dengan data teks
bow_transformer = CountVectorizer(analyzer=to_lemmas).fit(df['text'])
# Menampilkan jumlah kata unik dalam kosakata
print(len(bow_transformer.vocabulary_))

2114


In [11]:
# Mengambil teks ulasan ke-4 (indeks 3)
review1 = df['text'][3]
print(review1)
#to check 3rd document/review in collection/database

i liked the Da Vinci Code a lot.


In [12]:
# Mengubah teks menjadi representasi Bag of Words (vektor fitur)
bow = bow_transformer.transform([review1])
print(bow)
bow.shape  # Menampilkan dimensi vektor

  (0, 42)	1
  (0, 369)	1
  (0, 458)	1
  (0, 950)	1
  (0, 1123)	1
  (0, 1152)	1
  (0, 1838)	1
  (0, 1977)	1


(1, 2114)

In [13]:
# Melihat kata yang ada di indeks 372 dari kosakata
print(bow_transformer.get_feature_names_out()[372])
#to check 372nd word in collection

code-other


In [14]:
# Mengubah seluruh dataset teks menjadi representasi Bag of Words
review_bow = bow_transformer.transform(df['text'])
print('sparse matrix shape:', review_bow.shape)
print('number of non-zeros:', review_bow.nnz)  # Jumlah nilai non-nol dalam matriks
print('sparsity: %.2f%%' % (100.0 * review_bow.nnz / (review_bow.shape[0] * review_bow.shape[1])))  # Persentase sparsitas

sparse matrix shape: (6931, 2114)
number of non-zeros: 71297
sparsity: 0.49%


Tf-idf Vectorizer

In [15]:
# Mengubah representasi Bag of Words menjadi TF-IDF
# TF-IDF memberikan bobot lebih tinggi pada kata yang unik dan jarang muncul
tfidf_transformer = TfidfTransformer().fit(review_bow)
review_tfidf = tfidf_transformer.transform(review_bow)
review_tfidf.shape  # Menampilkan dimensi matriks TF-IDF

(6931, 2114)

In [16]:
# Membagi data menjadi data latih (80%) dan data uji (20%)
text_train, text_test, liked_train, liked_test = train_test_split(df['text'], df['liked'], test_size=0.2)
print(len(text_train), len(text_test), len(text_train), len(text_test))

5544 1387 5544 1387


In [17]:
# Membuat pipeline pemrosesan yang terdiri dari:
# 1. CountVectorizer (Bag of Words)
# 2. TfidfTransformer (TF-IDF)
# 3. SVC (Support Vector Classification)
pipeline_svm = Pipeline([
    ('bow', CountVectorizer(analyzer=to_lemmas)),
    ('tfidf', TfidfTransformer()),
    ('classifier', SVC()),
])

In [18]:
# Mendefinisikan parameter yang akan dieksplorasi untuk optimasi model
param_svm = [
    {'classifier__C': [1, 10, 100, 1000], 'classifier__kernel': ['linear']},
    {'classifier__C': [1, 10, 100, 1000], 'classifier__gamma': [0.001, 0.0001], 'classifier__kernel': ['rbf']},
]

SVM

In [19]:
# Melakukan Grid Search untuk menemukan parameter terbaik
grid_svm = GridSearchCV(
    pipeline_svm,  # object used to fit the data
    param_grid=param_svm,
    refit=True,  # fit using all data, on the best detected classifier
    n_jobs=-1,  # number of cores to use for parallelization; -1 for "all cores" i.e. to run on all CPUs
    scoring='accuracy',  # optimizing parameter
    cv=StratifiedKFold(n_splits=5),  # 5-fold cross validation dengan stratifikasi
)

In [20]:
# Melatih model dengan grid search
get_ipython().run_line_magic('time', 'classifier = grid_svm.fit(text_train, liked_train) # find the best combination from param_svm')
print(classifier.cv_results_)

CPU times: user 1.08 s, sys: 242 ms, total: 1.32 s
Wall time: 18.6 s
{'mean_fit_time': array([5.58978715, 5.48989902, 3.09671469, 1.2876081 , 2.84231939,
       2.80967612, 2.29692588, 2.7356513 , 1.35873237, 2.24772377,
       1.32600689, 1.13920345]), 'std_fit_time': array([0.24242936, 0.14841841, 2.25307242, 0.07066413, 0.17707761,
       0.24781218, 0.07171164, 0.03329448, 0.06873422, 0.05550277,
       0.050977  , 0.23360295]), 'mean_score_time': array([0.31283374, 0.30499916, 0.32684546, 0.31812096, 0.57820616,
       0.59130044, 0.53040872, 0.59727974, 0.34178147, 0.49111223,
       0.32925968, 0.23081779]), 'std_score_time': array([0.01982824, 0.01719803, 0.02459215, 0.03880559, 0.03221084,
       0.01272352, 0.03966375, 0.02234449, 0.01139661, 0.01909675,
       0.01538095, 0.05407407]), 'param_classifier__C': masked_array(data=[1, 10, 100, 1000, 1, 1, 10, 10, 100, 100, 1000, 1000],
             mask=[False, False, False, False, False, False, False, False,
                   F

In [21]:
# Mengevaluasi performa model pada data uji
print(classification_report(liked_test, classifier.predict(text_test)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       584
           1       1.00      0.99      0.99       803

    accuracy                           0.99      1387
   macro avg       0.99      0.99      0.99      1387
weighted avg       0.99      0.99      0.99      1387



In [22]:
# Contoh prediksi untuk kalimat positif
print(classifier.predict(["the vinci code is awesome"])[0])

1


In [23]:
# Contoh prediksi untuk kalimat negatif
print(classifier.predict(["the vinci code is bad"])[0])

0


In [24]:
# Implementasi fungsi kernel Gaussian untuk SVM
def gaussKernel(x1, x2, sigma):
    ss = np.power(sigma, 2)
    norm = (x1-x2).T.dot(x1-x2)
    return np.exp(-norm/(2*ss))

# Contoh penggunaan kernel Gaussian
x1 = np.array([1, 2, 1])
x2 = np.array([0, 4, -1])
sigma = 2
gaussKernel(x1, x2, sigma)  # Menghitung nilai kernel Gaussian antara dua vektor

0.32465246735834974